In [17]:
import requests
import json
import pandas as pd
import time
import twstock
import pymysql

In [18]:
conn=pymysql.connect(host='localhost',  # 主機位置
                       user='rock',       # 使用者帳號
                       password='A108-426a',   # 使用者密碼
                       db='Airocks',    # 資料庫名稱
                       port=3306,         # 連接阜
                       charset='utf8')    # 使用編碼

cursor = conn.cursor()

In [19]:
twses = twstock.twse
twses_data = pd.DataFrame()

for key in twses:
    data = twses[key]
    if data.CFI == 'ESVUFR':
        twselist = [data.name, data.code, data.ISIN, data.start, data.market, data.group, data.CFI]
        df = pd.DataFrame([twselist],columns=['name', 'code','ISIN','start','market','group','CFI'])
        twses_data=pd.concat([twses_data,df],axis=0,ignore_index=True)

In [20]:
uids=twses_data.code.tolist()

In [21]:
stock=pd.DataFrame()

for uid in uids:
    url='https://ws.api.cnyes.com/ws/api/v1/charting/history?resolution=D&symbol=TWS:'+uid+':STOCK&from=1606694400&to=1290902400'
    r=requests.get(url)
    wdata=r.json()['data']
    wdata.pop('s')
    wdata.pop('quote')
    wdata.pop('session')
    wdata.pop('nextTime')
    colname={
        't':'time',
        'o':'open',
        'h':'high',
        'l':'low',
        'c':'close',
        'v':'volume',
    }
    df=pd.DataFrame.from_dict(wdata)
    df=df.rename(columns=colname,inplace=False)
    df['time']=df['time'].apply(lambda x : time.strftime("%Y-%m-%d", time.localtime(x)))
    print(len(df))
    sql = "INSERT INTO Stocks_data(`StocksId`, `Date`, `Open`, `High`, `Low`, `Close`, `Volume`) VALUES "
    for i in range(len(df)):
        data = df.loc[i]
        if df.loc[0]['time'] != data['time']:
            sql+=','
        sql+="('%s', '%s', '%f', '%f', '%f', '%f', '%f')" % (uid, data['time'], data['open'], data['high'], data['low'], data['close'], data['volume'])    
    try:
        # 輸入 SQL指令
        cursor.execute(sql)
        # 執行 SQL指令
        conn.commit()
        print(uid+' Sccess!!')
    except:
        # 錯誤時重新執行
        conn.rollback()